# Fit, Optimize and Evaluate A Classifier


## Background

Now that we have training features extracted (and filtered if needed), we can train a machine learning model. There are various classification models available, and deciding which one to pick depends on the classification task at-hand. The table below provides a useful summary of the pros and cons of different models (all of which are available through [scikit-Learn](https://scikit-learn.org/stable/)). This sckit-learn [cheat sheet](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) may also help. _Table 1: Some of the pros and cons of different classifiers available through scikit-learn_
<img align="center" src="../../../Supplementary_data/Scalable_machine_learning/classifier_pro_cons.png" width="700">


In this example we are using a Random Forest classifier for the land cover classification task. We'll split the data into a training and testing set and use k-fold cross-validation to estimate the prediction ability of our model. To make full use of the limited number of training dara, here we will fit the final model on _all_ the training data.

## Description

This notebook runs through evaluating, optimizing, and fitting a machine learning classifier (in the default example, a Random Forest model is used). The steps are as follows:

1. Import training features
2. Optimize the hyperparameters of the model
3. Fit a model to all the training data using the best hyperparameters identified in the previous step
4. Calculate an unbiased performance estimate via cross-validation
5. Save the model to disk for use in the subsequent notebook

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

## Load packages

In [ ]:
%matplotlib inline
import os
import datacube
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score,cohen_kappa_score,confusion_matrix,ConfusionMatrixDisplay,balanced_accuracy_score
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score
from joblib import dump

## Analysis parameters

* `training_features_path`: The path to the file containing training features we extracted through previous module `0_Extract_Training_Features.ipynb` or filtered through `1_Filter_Training_Data.ipynb`.
* `dict_map` : A dictionary map of class names corresponding to class/pixel values.
* `class_attr`: This is the name of column in your shapefile/geojson file attribute table that contains the class labels. **The class labels must be integers.**

In [ ]:
training_features_path='Results/Mozambique_training_features_filtered.geojson'
dict_map={11:'Tree crops',12:'Field crops',21:'Forest plantations',31:'Grassland',
                 41:'Aquatic or regularly flooded herbaceous vegetation',44:'Water body',
                 51:'Settlements',61:'Bare soils',70:'Mangrove',71:'Mecrusse',
                72:'Broadleaved (Semi-) evergreen forest',74:'Broadleaved (Semi-) deciduous forest',75:'Mopane'} # dictionary of class names corresponding to pixel values
class_attr = 'LC_Class_I' # class label in integer format

## Import training data
We now load the training features file using `geopandas`. The dataframe should contain columns `class_attr` identifying class labels and the bi-monthly geomedians of the nine spectral bands and NDVI that we extracted through previous module:

In [ ]:
training_features= gpd.read_file(training_features_path) # Load training features
training_features.head() # Plot first five rows

For the following analysis we only need these features so we abandon the other columns, e.g. coordinates and geometry attributes.

In [ ]:
# drop columns not for analysis
df_training_data=training_features.iloc[:,:-5]

To check the number of samples for each class, we can plot a histogram by class. Note that the training features only contain integer class values, we retrive the class names using the `dict_map`:

In [ ]:
# count numbers of classes
class_counts=df_training_data[class_attr].value_counts()

# get class names
class_indices=class_counts.index
class_legends=[dict_map[class_indices[i]] for i in range(len(class_indices))]

# plot training samples distribution
plt.figure(figsize=(15,5))
ax=plt.bar(class_legends,height=class_counts.to_numpy())
plt.bar_label(ax)
plt.xticks(rotation=30, ha='right')
plt.gca().set_ylabel('Number of Training Samples')
plt.gca().set_xlabel('Land Cover Class')

> Note: One of the common problems with training data is **class imbalance**. This can occur when one of your classes is relatively rare and therefore the rare class will comprise a smaller proportion of the training set. When imbalanced data is used, it is common that the final classification will under-predict less abundant classes relative to their true proportion. If you think your data is imbalanced, you may consider adding more training samples for under-represented classes or removing samples for over-represented classes.

## Optimize hyperparameters

Machine learning models require certain 'hyperparameters': model parameters that can be tuned to increase the prediction ability of a model. Finding the best values for these parameters is a 'hyperparameter search' or an 'hyperparameter optimization'.

To optimize the parameters in our model, we use [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to exhaustively search through a set of parameters and determine the combination that will result in the highest accuracy based upon the accuracy metric defined. We first need to define a grid of parameters to be used in the optimization:
* `grid_parameters`: a dictionary of model specific parameters to search through during hyperparameter optimization.

> **Note**: the parameters in the `grid_parameters` object depend on the classifier being used. The default example is set up for a Random Forest classifier. To adjust the parameters to suit a different classifier, look up the important parameters under the relevant [sklearn documentation](https://scikit-learn.org/stable/supervised_learning.html).

In [ ]:
# Create the parameter grid based on the results of random search 
grid_parameters={'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 200, num = 3)],
                 'max_features': ['sqrt', 'log2'],'max_samples':[0.5]}

We now start searching for the optimal hyperparameters using `GridSearchCV`. It may take a while depending on your searching space. Here we use k-fold splitting strategy for the search, which we will cover later on.

In [ ]:
# initialise a random forest classifier
rf = RandomForestClassifier()

# stratified K-fold splitting strategy for grid search
cv=model_selection.StratifiedKFold(n_splits=5,shuffle=True,random_state=0) 

# grid search to find optimal random forest classifier hyperparameters
print('Grid searching best hyper-parameters...')
grid_search=model_selection.GridSearchCV(estimator = rf,param_grid=grid_parameters,cv=cv,n_jobs=-1)

#convert variable names into sci-kit learn nomenclature
X = df_training_data.to_numpy()[:,1:]
y = df_training_data.to_numpy()[:,0]

# fit to all the data
grid_search.fit(X,y)
print('Optimal parameters: \n',grid_search.best_params_)

## Fit the model

Using the best parameters from our hyperparmeter optimization search, we now fit our model on all the data:

In [ ]:
rf = RandomForestClassifier(**grid_search.best_params_, random_state=1, n_jobs=-1)
rf.fit(X,y)

## Determine feature importance

Here we extract classifier estimates of the relative importance of each feature for training a Random Forest classifier. This is useful for potentially selecting a subset of input bands/variables for model training/classification (i.e. optimising feature space). However, in this case, we are not selecting a subset of features, but rather just trying to understand the importance of each feature. This can help us not only understand our classes better (e.g. what kinds of measurements are more important in predicting the land cover classes), but can also lead to further improvements to the model. 

Sklearn has good documentation on different methods for [feature selection](https://scikit-learn.org/stable/modules/feature_selection.html).

Results will be presented in ascending order with the most important features listed last. Importance is reported as a relative fraction between 0 and 1. These importances are based on how much a given feature, on average, decreases the weighted Gini impurity.


In [ ]:
# plot feature importance
plt.figure(figsize=(30,5))
order=np.argsort(rf.feature_importances_)
plt.bar(x=np.array(df_training_data.columns[1:])[order],height=rf.feature_importances_[order],width=0.5,align='center')
plt.xticks(rotation='vertical',fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Variable', fontsize=20)
plt.ylabel('Importance', fontsize=20)

Observe the above figure and get an idea of what bands/features are most important for discriminating different classes in our selected area. 

## Evaluate performance via cross-validation

K-fold [cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) is a statistical method used to estimate the performance of machine learning models when making predictions on data not used during training.  It is a popular method because it is conceptually straightforward and because it generally results in a less biased or less optimistic estimate of the model skill than other methods, such as a simple train/test split. This procedure can be used both when optimizing the hyperparameters of a model on a dataset, and when comparing and selecting a model for the dataset. 

Since our training dataset is small and to make sure each class has samples in the training set, in this example the `StratifiedKFold` splitting is used. Stratified k-fold is a variation of k-fold which returns stratified folds: each set contains approximately the same percentage of samples of each target class as the complete set. We define 5 folds splitting, which divides the dataset into 5 sets:

In [ ]:
# stratified K-fold splitting
skf=model_selection.StratifiedKFold(n_splits=5,shuffle=True,random_state=1)

We now use `cross_val_score` function which calculates an accuracy score that show how well our classifier is doing at recognising points in each test split through cross validation. The default example is set up to calculate the `accuracy_score` or Overall Accuracy (OA), which is the fraction of correct predictions for all the samples. The OA will be between 0 and 1, with a value of 1 indicating a perfect score. There are other scoring paramters that you can choose [here](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter).

In [ ]:
# calculate overall accurcy through cross validation
overall_acc=model_selection.cross_val_score(rf,X,y,cv=skf,scoring='accuracy')
print('Overall accuracy from cross validation: ',np.mean(overall_acc))

To show how the prediction performs on each class, the example will calculate the predictions from cross validation:

In [ ]:
# predict through cross validation
predictions=model_selection.cross_val_predict(rf,X,y,cv=skf)

Using the predicted labels, we can calculate class-wise scores, such as precision and recall. The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative. The best value is 1 and the worst value is 0. We use the function `precision_score` for the calculation:

In [ ]:
precision=precision_score(y,predictions,labels=list(dict_map.keys()),average=None)
print('Precision for each class: \n',dict(zip([dict_map[value] for value in list(dict_map.keys())],np.around(precision,3))))

Similarly we can calculate the recall scores using the function `recall_score`. The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

In [ ]:
recall=recall_score(y,predictions,labels=list(dict_map.keys()),average=None)
print('Recall for each class: \n',dict(zip([dict_map[value] for value in list(dict_map.keys())],np.around(precision,3))))

To further inspect the actual numbers of predictions for each class we calculate and plot the `confusion_matrix`. By definition, entry *i, j* in a confusion matrix is the number of observations actually in group *i*, but predicted to be in group *j*:

In [ ]:
# generate and visualise confusion matrix
cm=confusion_matrix(y, predictions)
display_labels=[dict_map[rf.classes_[i]] for i in range(len(rf.classes_))]
disp=ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=display_labels)
fig, ax = plt.subplots(figsize=(15,15))
disp.plot(ax=ax)
ax.set_title('Confusion Matrix')

## Save the model
We now export the trained classifier as a binary`.joblib` file, which will allow for importing the model in the subsequent module.

In [ ]:
dump(rf, 'Results/Mozambique_RF_model.joblib')